**Table of contents**<a id='toc0_'></a>    
- [seq2se2 실습](#toc1_1_)    
  - [데이터 로딩](#toc1_2_)    
  - [전처리](#toc1_3_)    
  - [데이터 변환](#toc1_4_)    
  - [데이터 분리 : train & test](#toc1_5_)    
  - [데이터로더](#toc1_6_)    
  - [모델링](#toc1_7_)    
  - [모델 로딩](#toc1_8_)    
  - [동작시키기 (test 모드)](#toc1_9_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[seq2se2 실습](#toc0_)
- 영어->프랑스어 변역기
- python -v 3.8.19
- pytorch

In [1]:
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import numpy as np
import pandas as pd
import torch
from collections import Counter
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset


## <a id='toc1_2_'></a>[데이터 로딩](#toc0_)

In [28]:
import requests
import os
# User-Agent => Python의 기본 requests 요청을 차단 방지
# timeout => 네트워크 문제로 인한 응답 지연 확인
# 파일을 wb(바이너리 모드)로 저장해서 손상 방지

url = "http://www.manythings.org/anki/fra-eng.zip"
filename = "./attach/fra-eng.zip"

# User-Agent 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 파일 다운로드
response = requests.get(url, headers=headers, stream=True, timeout=10)

if response.status_code == 200:
    with open(filename, "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    print("✅ Download complete!")
else:
    print(f"❌ 다운로드 실패: HTTP {response.status_code}")


✅ Download complete!


In [29]:
# 압축 풀기
import zipfile
import os

with zipfile.ZipFile("./attach/fra-eng.zip", "r") as zip_ref:
    zip_ref.extractall("attach")  # 원하는 경로로 변경 가능

print("Unzip complete!")


Unzip complete!


## <a id='toc1_3_'></a>[전처리](#toc0_)

In [30]:
# 전처리

def unicode_to_ascii(s):
    # 프랑스어 악센트 삭제
    # 예시 : 'déjà diné' -> deja dine
    return ''.join(c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c)!='Mn')


In [31]:
def preprocess_sentence(sent):
    # 악센트 삭제 함수 호출
    sent=unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백 생성
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    # 다수 개의 공백을 하나의 공백으로 치환
    sent = re.sub(r"\s+", " ", sent)
    return sent


In [32]:
num_samples = 1000

def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("./attach/fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence(src_line).split()]
      
      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_in = [w for w in ("<sos> " + tar_line).split()]
      tar_line_out = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target
# 디코더 입력 데이터를 생성한 이유
# 모델구조는 현재 시점의 디코더 셀의 입력은 오직 이전 디코더 셀의 출력을 입력으로 받는다
# 하지만, 학습과정에서는 학습 효율을 높이기 위해 이전 디코더 셀의 예측값 대신 실제값을 넣어주므로
# 별로도 디코터 입력 데이터도 만든 것이다


In [33]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))


전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


In [34]:
# 데이터 확인
# ['go', '.'] => 문장1개로 간주하자
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


## <a id='toc1_4_'></a>[데이터 변환](#toc0_)

In [35]:
# 단어집합 생성
# -----------------
# word <-> index
# 빈도순으로 정수 정렬
# 0번<PAD>, 1번<UNK>, 2번<빈도수 가장 많은 단어> ...

def build_vocab(sents):
    word_list = []
    for sent in sents:
        for word in sent:
            word_list.append(word)
    # 빈도수 계산 및 정렬
    word_counts = Counter(word_list)
    vocab = sorted(word_counts, key=word_counts.get, reverse=True)

    word_to_index={}
    word_to_index['<PAD>']=0
    word_to_index['<UNK>']=1
    for index, word in enumerate(vocab):
        word_to_index[word] = index+2

    return word_to_index  

In [36]:
# word -> index 
src_to_index = build_vocab(sents_en_in)
tar_to_index = build_vocab(sents_fra_in + sents_fra_out)

src_vocab_size = len(src_to_index)
tar_vocab_size = len(tar_to_index)

print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))


# index -> word
index_to_src = {idx:word for word,idx in src_to_index.items() }
index_to_tar = {idx:word for word,idx in tar_to_index.items() }
print(index_to_src)

영어 단어 집합의 크기 : 292, 프랑스어 단어 집합의 크기 : 635
{0: '<PAD>', 1: '<UNK>', 2: '.', 3: '!', 4: 'i', 5: 'it', 6: 'get', 7: 'go', 8: 'm', 9: 'away', 10: 'out', 11: 'me', 12: '?', 13: 'up', 14: 'tom', 15: 'be', 16: 'lost', 17: 'on', 18: 'come', 19: 'ahead', 20: 'we', 21: 'run', 22: 'down', 23: 's', 24: 'beat', 25: 'won', 26: 'terrific', 27: 'you', 28: 'back', 29: 'this', 30: 'try', 31: 'no', 32: 'who', 33: 'relax', 34: 'way', 35: 'fair', 36: 'nice', 37: 'us', 38: 'how', 39: 'look', 40: 'help', 41: 'hold', 42: 'calm', 43: 'off', 44: 'forget', 45: 'see', 46: 'got', 47: 'him', 48: 'he', 49: 'shut', 50: 'fun', 51: 'leave', 52: 'don', 53: 't', 54: 'stop', 55: 'wait', 56: 'call', 57: 'take', 58: 'did', 59: 'use', 60: 'sit', 61: 'excuse', 62: 'hello', 63: 'now', 64: 'in', 65: 'left', 66: 'paid', 67: 'home', 68: 'hang', 69: 'll', 70: 'kill', 71: 'open', 72: 'am', 73: 'brief', 74: 'good', 75: 'hurry', 76: 'ok', 77: 'really', 78: 'ask', 79: 'cool', 80: 'goodbye', 81: 'sad', 82: 'drive', 83: 'push', 84: 'wake

In [37]:
# 정수인코딩

def texts_to_sequences(sents, word_to_index):
    encoded_X_data = []
    for sent in tqdm(sents):
        index_sequences = []
        for word in sent:
            try:
                index_sequences.append(word_to_index[word])
            except KeyError:
                index_sequences.append(word_to_index['<UNK>'])
        encoded_X_data.append(index_sequences)
    return encoded_X_data

In [38]:
encoder_input = texts_to_sequences(sents_en_in, src_to_index)
decoder_input = texts_to_sequences(sents_fra_in, tar_to_index)
decoder_target = texts_to_sequences(sents_fra_out, tar_to_index)

# 상위 5개의 샘플에 대해서 정수 인코딩 전, 후 문장 출력
# 인코더 입력이므로 <sos>나 <eos>가 없음
for i, (item1, item2) in zip(range(5), zip(sents_en_in, encoder_input)):
    print(f"Index: {i}, 정수 인코딩 전: {item1}, 정수 인코딩 후: {item2}")


100%|██████████| 1000/1000 [00:00<00:00, 984116.38it/s]

Index: 0, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [7, 2]
Index: 1, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [7, 2]
Index: 2, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [7, 2]
Index: 3, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [7, 2]
Index: 4, 정수 인코딩 전: ['hi', '.'], 정수 인코딩 후: [122, 2]


In [39]:
# 패딩
# 최대 길이 값이 주어지지 않을 경우 데이터 내 최대 길이로 패딩
def pad_sequences(sentences, max_len=None):
    if max_len is None:
        max_len = max([len(sentence) for sentence in sentences])

    features = np.zeros((len(sentences), max_len), dtype=int)
    for index, sentence in enumerate(sentences):
        if len(sentence)!=0:
            features[index, :len(sentence)] = np.array(sentence)[:max_len]
    return features

In [40]:
encoder_input = pad_sequences(encoder_input)
decoder_input = pad_sequences(decoder_input)
decoder_target = pad_sequences(decoder_target)

print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)


인코더의 입력의 크기(shape) : (1000, 4)
디코더의 입력의 크기(shape) : (1000, 10)
디코더의 레이블의 크기(shape) : (1000, 10)


## <a id='toc1_5_'></a>[데이터 분리 : train & test](#toc0_)

In [41]:
# 데이터 분리

# 데이터 섞기
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

print([index_to_src[word] for word in encoder_input[5]])
print([index_to_tar[word] for word in decoder_input[5]])
print([index_to_tar[word] for word in decoder_target[5]])




랜덤 시퀀스 : [498  83 403 123 298 169  91 839 318 970 361 161 446 663 617 951 644 576
 860 883  40 864 815 140 261 151 335 439 499 534 459 481  84  28 639 506
 952 445 693 433 956 157 147 101  67 237 637 184 525 973 796  58 721 932
 172 175 602 665  76 280 792 819 155 892 756 824 806 723 284 670  75 369
 548 142 319 366 825 278 156 759 251 505 255 642 501  66 372 356 376  22
 486 881 735 712 734 465 544 696 204 874 223 804 520 180 305 390 222 803
 885   1 423 504  95 758 206  27 540  48 209 490 492 488  35 441 386 457
 777 272 711  59 776 573 160 473 943 391  19 610  47  94 256 477 683 333
 733 958 782 914 771 246 516 212 969 182 826 148 364 886 307 808 676 852
 722 347 717 991 672 558 719 997 349 300 828 330  72 210  50 744 352  77
 774 944 177 235 196 800 311 884 303 994 757  11  45 961 456 560 811 679
 833 873 383 487 813  18 461 135 414 917  21 580 786 664 613 767 764  93
 768 667 596 408 217 468 750 628 578 264 902 502 341  89 214 631 130 221
 779 281 922  98 569 714 497 437 344 791 7

In [42]:
n_of_val = int(num_samples*0.1)
print('검증 데이터의 개수 :',n_of_val)

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]


검증 데이터의 개수 : 100


In [43]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)


훈련 source 데이터의 크기 : (900, 4)
훈련 target 데이터의 크기 : (900, 10)
훈련 target 레이블의 크기 : (900, 10)
테스트 source 데이터의 크기 : (100, 4)
테스트 target 데이터의 크기 : (100, 10)
테스트 target 레이블의 크기 : (100, 10)


## <a id='toc1_6_'></a>[데이터로더](#toc0_)

In [44]:
encoder_input_train_tensor = torch.tensor(encoder_input_train, dtype=torch.long)
decoder_input_train_tensor = torch.tensor(decoder_input_train, dtype=torch.long)
decoder_target_train_tensor = torch.tensor(decoder_target_train, dtype=torch.long)

encoder_input_test_tensor = torch.tensor(encoder_input_test, dtype=torch.long)
decoder_input_test_tensor = torch.tensor(decoder_input_test, dtype=torch.long)
decoder_target_test_tensor = torch.tensor(decoder_target_test, dtype=torch.long)

# 데이터셋 및 데이터로더 생성
batch_size = 128

train_dataset = TensorDataset(encoder_input_train_tensor, decoder_input_train_tensor, decoder_target_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = TensorDataset(encoder_input_test_tensor, decoder_input_test_tensor, decoder_target_test_tensor)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)


## <a id='toc1_7_'></a>[모델링](#toc0_)

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim

In [46]:

# 모델 구조
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embedding_size, hidden_size):
        super(Encoder, self).__init__() # 부모 클래스의 생성자 호출
        self.embedding = nn.Embedding(src_vocab_size, embedding_size, padding_idx=0)
        self.lstm = nn.LSTM(embedding_size, hidden_size, batch_first=True)

    def forward(self, x):
        # x.shape == (batch,time,embedding) 
        x = self.embedding(x)
        # hidden.shape == (1,batch,hidden), cell.shape == (1,batch,hidden)
        _, (hidden,cell) = self.lstm(x)  # (전체시점 히든, (마지막시점 히든,마지막시점 셀))
        # 인코더 출력은 hidden state, cell state
        return hidden, cell
    
class Decoder(nn.Module):
    def __init__(self, tar_vocab_size, embedding_size, hidden_size):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(tar_vocab_size, embedding_size, padding_idx=0)
        self.lstm = nn.LSTM(embedding_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, tar_vocab_size)

    def forward(self, x, hidden, cell):
        # x.shape == (batch,time,embedding)
        x = self.embedding(x)

        # 디코더의 LSTM으로 인코더의 hidden state, cell state 전달
        # output.shape == (batch,time,hidden)
        # hidden.shape == (1,batch,hidden) 마지막시점이라서 T=1
        # cell.shape == (1,batch,hidden)   마지막시점이라서 T=1
        output, (hidden,cell) = self.lstm(x, (hidden, cell))

        # output.shape == (batch,time,vocab)
        output = self.fc(output)

        # 디코더의 출력은 예측값, hidden state, cell state
        return output, hidden, cell
    
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq,self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        hidden, cell = self.encoder(src)

        # 학습 중에는 디코더의 출력 중 오직 output만 사용
        # 디코더 입력과정에서 실제값(정답)을 사용하기 때문에
        # lstm계층에서 hidden state와 cell state를 전파할 필요 없음
        # 반면, 실제 모델사용할때는 
        # 디코더 입력과정에서 이전 디코더의 예측값을 사용하기 때문에
        # lstm계층에서 hidden state와 cell state를 계속 흘려야 함
        output, _, _ = self.decoder(trg, hidden, cell)
        return output


In [47]:
# 모델선언
embedding_size = 256
hidden_size = 256

encoder= Encoder(src_vocab_size, embedding_size, hidden_size)
decoder= Decoder(tar_vocab_size, embedding_size, hidden_size)
model = Seq2Seq(encoder,decoder)

loss_function = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters())

# ignore_index=0 : 손실 계산 시 특정 레이블(0인 클래스)을 무시
# 예) 정답 레이블이 [1, 0, 2, 0]이고, 모델의 예측이 [0.2, 0.5, 0.1, 0.4]일 때, 
# 0으로 패딩된 위치는 손실 계산에 포함되지 않음
print(model)


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(292, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(635, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=635, bias=True)
  )
)


In [48]:
# 학습 설정
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(292, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(635, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=635, bias=True)
  )
)

In [49]:
def evaluation(model, dataloader, loss, device):
    # 평가모드 설정
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_count = 0

    with torch.no_grad(): # 기울기 계산 비활성화
        for encoder_input, decoder_input, decoder_target in dataloader:
            encoder_input = encoder_input.to(device)
            decoder_input = decoder_input.to(device)
            decoder_target = decoder_target.to(device)

            # 순방향 전파
            # output.shape == (batch, time, tar_vocab)
            output = model(encoder_input, decoder_input)

            # 손실 계산
            # shape 변환: 예측값.shape => 2D (batch x time, vocab) 각 vacob의 확률값
            # shape 변환: 실제값.shape => 1D ( ,batch x time) 정수값
            # ????? 손실계산 방법이 잘 이해XXX??????????????
            loss = loss_function(output.view(-1,output.size(-1)), decoder_target.view(-1)) 
            total_loss += loss.item()

            # 정확도 계산(패딩 토큰 제외)
            mask = decoder_target != 0
            total_correct += ((output.argmax(axis=-1) == decoder_target) * mask).sum().item()
            total_count += mask.sum().item()
    

    return total_loss / len(dataloader), total_correct / total_count
    # 평균손실은 총손실은 데이터로더의 배치수로 나눈다
    # 정확도는 총정확도를 총 토큰 수로 나눈다



In [51]:
best_val_loss = float('inf')

for epoch in range(num_epochs):
    # 훈련모드 설정
    model.train()

    for encoder_input, decoder_input, decoder_target in train_dataloader:
        encoder_input = encoder_input.to(device)
        decoder_input = decoder_input.to(device)
        decoder_target = decoder_target.to(device)

        # 순전파
        output = model(encoder_input, decoder_input)

        # 손실
        loss = loss_function(output.view(-1, output.size(-1)), decoder_target.view(-1))

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        


    # 평가
    train_loss, train_acc = evaluation(model, train_dataloader, loss_function, device)
    valid_loss, valid_acc = evaluation(model, valid_dataloader, loss_function, device)

    print(f'Epoch: {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.4f}')

    # 검증 손실이 최소일 때 체크포인트 저장
    if valid_loss < best_val_loss:
        print(f'Validation loss improved from {best_val_loss:.4f} to {valid_loss:.4f}. 체크포인트를 저장합니다.')
        best_val_loss = valid_loss
        torch.save(model.state_dict(), './attach/best_model_checkpoint.pth')
    

Epoch: 1/10 | Train Loss: 2.2619 | Train Acc: 0.5521 | Valid Loss: 2.9940 | Valid Acc: 0.4964
Validation loss improved from inf to 2.9940. 체크포인트를 저장합니다.
Epoch: 2/10 | Train Loss: 2.1673 | Train Acc: 0.5750 | Valid Loss: 2.9371 | Valid Acc: 0.5206
Validation loss improved from 2.9940 to 2.9371. 체크포인트를 저장합니다.
Epoch: 3/10 | Train Loss: 1.9995 | Train Acc: 0.5781 | Valid Loss: 2.8878 | Valid Acc: 0.5157
Validation loss improved from 2.9371 to 2.8878. 체크포인트를 저장합니다.
Epoch: 4/10 | Train Loss: 1.9576 | Train Acc: 0.5889 | Valid Loss: 2.8361 | Valid Acc: 0.5327
Validation loss improved from 2.8878 to 2.8361. 체크포인트를 저장합니다.
Epoch: 5/10 | Train Loss: 1.7623 | Train Acc: 0.6070 | Valid Loss: 2.7838 | Valid Acc: 0.5424
Validation loss improved from 2.8361 to 2.7838. 체크포인트를 저장합니다.
Epoch: 6/10 | Train Loss: 1.7178 | Train Acc: 0.6211 | Valid Loss: 2.7436 | Valid Acc: 0.5448
Validation loss improved from 2.7838 to 2.7436. 체크포인트를 저장합니다.
Epoch: 7/10 | Train Loss: 1.6908 | Train Acc: 0.6348 | Valid Loss: 

## <a id='toc1_8_'></a>[모델 로딩](#toc0_)

In [52]:
model.load_state_dict(torch.load('./attach/best_model_checkpoint.pth'))
model.to(device)
val_loss, val_accuracy = evaluation(model, valid_dataloader, loss_function, device)

print(f'Best model validation loss: {val_loss:.4f}')
print(f'Best model validation accuracy: {val_accuracy:.4f}')



Best model validation loss: 2.5940
Best model validation accuracy: 0.5642


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_34576\3434567499.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./attach/best_

In [54]:
print(tar_to_index['<sos>'])
print(tar_to_index['<eos>'])

3
4


## <a id='toc1_9_'></a>[동작시키기 (test 모드)](#toc0_)

In [55]:
# seq2seq는 훈련 과정(교사 강요)과 테스트 과정에서의 동작 방식이 다름
# 그래서 테스트 과정을 위해 모델을 다시 설계(디코더 수정)

# 1) 번역하고자 하는 입력 문장이 인코더로 입력되어 인코더의 마지막 시점의 은닉 상태와 셀 상태를 얻는다
# 2) 인코더의 은닉 상태와 셀 상태, 그리고 토큰 <sos>를 디코더로 전달
# 3) 디코더가 토큰 <eos>가 나올 때까지 다음 단어를 예측하는 행동을 반복

index_to_src = {i: w for w, i in src_to_index.items()}
index_to_tar = {i: w for w, i in tar_to_index.items()}

def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0):
            sentence = sentence + index_to_src[encoded_word]+' '
    return sentence

def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
            sentence = sentence + index_to_tar[encoded_word]+' '
    return sentence

print(encoder_input_test[25])
print(decoder_input_test[25])
print(decoder_target_test[25])

[156   2   0   0]
[  3 137   2   0   0   0   0   0   0   0]
[137   2   4   0   0   0   0   0   0   0]


In [56]:
# 테스트 단계에서는 디코더를 매 시점 별로 컨트롤
# 사용될 변수는 decoder_input
# unsquzzez(dim) : dim번째 차원에 차원추가

def decode_sequence(input_seq, model, max_output_len, src_vocab_size, tar_vocab_size, int_to_src, int_to_tar):
    # input_seq: 단어 1개의 특징벡터(embedding 벡터)
    encoder_input = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device) # 3D (배치1)

    # 인코더의 초기 상태 설정
    hidden, cell = model.encoder(encoder_input)

    # 시작 토큰 <SOS>를 디코더의 첫입력으로 설정
    # unsquzzez(0)은 배치 차원 추가 목적
    decoder_input = torch.tensor([3], dtype=torch.long).unsqueeze(0).to(device) # 2D
    decoded_tokens = []

    # for문을 도는 것 == 디코더의 각 시점
    # 디코더의 각 시점마다 이전 예측값을 decoder_input으로 전달
    print("input_seq: ", input_seq)
    for i in range(max_output_len):
        output, hidden, cell = model.decoder(decoder_input, hidden, cell)
        
        # 소프트맥스 회귀. 예측 단어의 인덱스
        output_token = output.argmax(dim=-1).item()
        
        # report
        print("report=> {}th | decoder_input=> {} |output_token=> {}".format(i,decoder_input, output_token))
        
        # 종료 토큰 <eos>
        if output_token == 4:
            print("decoded_tokens: ", decoded_tokens)
            break

        # 현재 시점의 예측값을 다음 시점의 입력으로 전달
        decoder_input = torch.tensor([output_token], dtype=torch.long).unsqueeze(0).to(device)

        
        # 각 시점의 단어(정수)는 decoded_tokens에 누적하였다가 최종 번역 시퀀스로 리턴
        decoded_tokens.append(output_token)


    result = ' '.join(int_to_tar[token] for token in decoded_tokens)
    print("result: ", result)
    print("-"*50) 
    return result



In [57]:
# 훈련데이터에 대해 샘플 결과 확인

for seq_index in [3,50,70,99]: 
    # -------- 데이터 단위 이해 --------------
    # 리스트[]로 묶인 건 문장 단위이고, 리스트 안의 원소는 단어(vocab) 단위
    #  ['go', '.'] 같은 문장을 단어로 쪼개어 인덱스화 해서 모은 것

    # 여기서 [3,50,100,300]의 각 원소는 사실 문장단위로
    # 번역 테스트할 문장을 리스트에 모아놓은 것
    # 원소3을 encoder_input_train에서 인덱싱하면 [ 4 65  2  0] 문장!!

    # encoder_input_train : 훈련에 사용된 문장들의 집합 (2D배열)
    # input_seq: 위 문장집합에서 해당하는 문장 추출한 것 (1D배열)
    print("seq_index: ", seq_index)
    input_seq = encoder_input_train[seq_index] 
    translated_text = decode_sequence(input_seq, model, 20, src_vocab_size, tar_vocab_size, index_to_src, index_to_tar)


    print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
    print("번역문장 :",translated_text)
    print("="*50)

seq_index:  3
input_seq:  [  4 221   2   0]
report=> 0th | decoder_input=> tensor([[3]]) |output_token=> 6
report=> 1th | decoder_input=> tensor([[6]]) |output_token=> 7
report=> 2th | decoder_input=> tensor([[7]]) |output_token=> 2
report=> 3th | decoder_input=> tensor([[2]]) |output_token=> 4
decoded_tokens:  [6, 7, 2]
result:  je suis .
--------------------------------------------------
입력문장 : i pass . 
정답문장 : je passe . 
번역문장 : je suis .
seq_index:  50
input_seq:  [203   5   2   0]
report=> 0th | decoder_input=> tensor([[3]]) |output_token=> 82
report=> 1th | decoder_input=> tensor([[82]]) |output_token=> 8
report=> 2th | decoder_input=> tensor([[8]]) |output_token=> 2
report=> 3th | decoder_input=> tensor([[2]]) |output_token=> 4
decoded_tokens:  [82, 8, 2]
result:  prenez le .
--------------------------------------------------
입력문장 : prove it . 
정답문장 : prouvez le . 
번역문장 : prenez le .
seq_index:  70
input_seq:  [125   2   0   0]
report=> 0th | decoder_input=> tensor([[3]]) |outpu

In [58]:
# 테스트트데이터에 대해 샘플 결과 확인

for seq_index in [3,50,70,99]: 
    # -------- 데이터 단위 이해 --------------
    # 리스트[]로 묶인 건 문장 단위이고, 리스트 안의 원소는 단어(vocab) 단위
    #  ['go', '.'] 같은 문장을 단어로 쪼개어 인덱스화 해서 모은 것

    # 여기서 [3,50,100,300]의 각 원소는 사실 문장단위로
    # 번역 테스트할 문장을 리스트에 모아놓은 것
    # 원소3을 encoder_input_train에서 인덱싱하면 [ 4 65  2  0] 문장!!

    # encoder_input_train : 훈련에 사용된 문장들의 집합 (2D배열)
    # input_seq: 위 문장집합에서 해당하는 문장 추출한 것 (1D배열)
    print("seq_index: ", seq_index)
    input_seq = encoder_input_test[seq_index] 
    translated_text = decode_sequence(input_seq, model, 20, src_vocab_size, tar_vocab_size, index_to_src, index_to_tar)


    print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
    print("번역문장 :",translated_text)
    print("="*50)

seq_index:  3
input_seq:  [ 15 285   2   0]
report=> 0th | decoder_input=> tensor([[3]]) |output_token=> 22
report=> 1th | decoder_input=> tensor([[22]]) |output_token=> 5
report=> 2th | decoder_input=> tensor([[5]]) |output_token=> 4
decoded_tokens:  [22, 5]
result:  soyez !
--------------------------------------------------
입력문장 : i pass . 
정답문장 : je passe . 
번역문장 : soyez !
seq_index:  50
input_seq:  [24  5  2  0]
report=> 0th | decoder_input=> tensor([[3]]) |output_token=> 74
report=> 1th | decoder_input=> tensor([[74]]) |output_token=> 2
report=> 2th | decoder_input=> tensor([[2]]) |output_token=> 4
decoded_tokens:  [74, 2]
result:  deguerpissez .
--------------------------------------------------
입력문장 : prove it . 
정답문장 : prouvez le . 
번역문장 : deguerpissez .
seq_index:  70
input_seq:  [ 14 147   2   0]
report=> 0th | decoder_input=> tensor([[3]]) |output_token=> 18
report=> 1th | decoder_input=> tensor([[18]]) |output_token=> 12
report=> 2th | decoder_input=> tensor([[12]]) |output